In [ ]:
import os
import sys
import zipfile
import subprocess
import pandas as pd
from datetime import date, timedelta
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("ml1.csv")
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace=True)
df.sort_index(inplace=True)
df2 = df.loc["2020":]
df3 = df2.drop(columns=["ma_60", "rsi", "macd", "volume", "dividends", "stock_splits", "marketcap", "gain_14", "loss_14", "rsi_14", "ema_12", "ema_26", "rsi_lag_5", "rsi_lag_10"])
df3.reset_index(inplace=True)


In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace=True)
df.sort_index(inplace=True)
df2 = df.loc["2020":]

In [ ]:
df3 = df2.drop(columns=["ma_60", "rsi", "macd", "volume", "dividends", "stock_splits", "marketcap", "gain_14", "loss_14", "rsi_14", "ema_12", "ema_26", "rsi_lag_5", "rsi_lag_10"])

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df3.head()

In [ ]:
from datetime import datetime, timedelta

top = pd.read_csv("top_20.csv")

future = datetime.today().date() + timedelta(days=30)
top["date"] = pd.to_datetime(future)

top.drop(columns=["close", "gain_predicted"], inplace=True)
top.rename(columns={"prediction": "close"}, inplace=True)

merged = pd.concat([df3, top], ignore_index=True)
merged.sort_values(by="date", inplace=True)

In [ ]:
merged

In [ ]:
merged.sort_values(by="date", inplace=True)
merged

In [ ]:
# plotting for test

co = "V"



plt.figure(figsize=(12, 8))
sns.set_theme(style="whitegrid")

sns.lineplot(data=df3[df3["company"] == co], x="date", y="bb_lower", label="10")
sns.lineplot(data=df3[df3["company"] == co], x="date", y="ma_20", label="moving average 20")
sns.lineplot(data=df3[df3["company"] == co], x="date", y="bb_upper", label="30")
sns.lineplot(data=df3[df3["company"] == co], x="date", y="close", label="Price")

plt.axhline(y=70)
plt.axhline(y=30)

plt.title(f'Moving Averages for {co}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=14)
plt.ylabel('Moving Average', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.legend(fontsize=12, loc='best')
plt.tight_layout()

In [1]:
import os
import sys
import zipfile
import subprocess
import pandas as pd
from datetime import datetime, date, timedelta
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [2]:
sys.path.append('/Users/jangfeller/Library/Mobile Documents/com~apple~CloudDocs/_Ironhack/projects/mid_project/app/utils/')
from functions import get_latest_master, create_ML1_df, create_ML1_df_training, run_xgb, plot_data


In [ ]:

step_1 = get_latest_master()
print("data downloaded")
ml1 = create_ML1_df(step_1)
ml1.to_csv("ml1.csv")
print("ML1 created")
ml1_training = create_ML1_df_training(step_1)
print("ML1 training created")
tipps = run_xgb(ml1, ml1_training)
tipps.to_csv("top_20.csv")
print("tipps ready")


In [3]:
step_1 = get_latest_master()


Dataset URL: https://www.kaggle.com/datasets/benjaminpo/s-and-p-500-with-dividends-and-splits-daily-updated
License(s): CC-BY-SA-4.0


100%|██████████| 156M/156M [00:13<00:00, 12.2MB/s] 


In [7]:
step_1

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,company
0,1999-11-18 00:00:00-05:00,27.466222,30.182664,24.146130,26.560743,62546380,0.0,0.0,A
1,1999-11-19 00:00:00-05:00,25.919361,25.957091,24.032946,24.372501,15234146,0.0,0.0,A
2,1999-11-22 00:00:00-05:00,24.938424,26.560743,24.183858,26.560743,6577870,0.0,0.0,A
3,1999-11-23 00:00:00-05:00,25.655259,26.334369,24.146126,24.146126,5975611,0.0,0.0,A
4,1999-11-24 00:00:00-05:00,24.221581,25.315703,24.146124,24.787506,4843231,0.0,0.0,A
...,...,...,...,...,...,...,...,...,...
4238170,2024-12-10 00:00:00-05:00,177.880005,178.509995,176.199997,176.710007,1678200,0.0,0.0,ZTS
4238171,2024-12-11 00:00:00-05:00,176.259995,178.380005,175.809998,177.169998,1782400,0.0,0.0,ZTS
4238172,2024-12-12 00:00:00-05:00,176.949997,179.699997,176.559998,178.839996,1936000,0.0,0.0,ZTS
4238173,2024-12-13 00:00:00-05:00,178.979996,181.850006,176.630005,178.179993,1650300,0.0,0.0,ZTS


In [4]:
ml1 = create_ML1_df(step_1)


In [5]:
ml1_training = create_ML1_df_training(step_1)

In [11]:
def run_xgb(df4, df4_training):

    """ creates df with top 20 predicted gains """

    def XGB_train_real():

        # short cleaning
        df_x = df4_training.copy().reset_index()
        df_x.drop(columns=["company", "sector", "subsector"], inplace=True)
        df_x.set_index("date", inplace=True)
        df_x.sort_index(inplace=True)

        # training with data only until training_end
        xg_df = df_x.copy()

        # X y split
        X = xg_df.drop(columns="price_30d")
        y = xg_df["price_30d"]

        # normalization
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)

        # train model
        xgbr = xgb.XGBRFRegressor()
        xgbr.fit(X_scaled, y)

        return xgbr



    def get_stocks():

        """
        Output: df with top 20 gainers
        """

        df_ga = df4.copy()
        df_ga.reset_index(inplace=True)
        df_ga["date"] = pd.to_datetime(df_ga["date"])
        gains_df = pd.DataFrame(columns=["company", "close", "prediction", "gain_predicted"])
        df_ga.drop(columns=["sector", "subsector"], inplace=True)
        df_ga.set_index("date", inplace=True)    
        df_ga.sort_index(inplace=True)

        yesterday = date.today() - timedelta(days=2)
        if not yesterday in df_ga:
            yesterday = date.today() - timedelta(days=3)
            if not yesterday in df_ga:
                yesterday = date.today() - timedelta(days=4)

        xg_df = df_ga.loc[str(yesterday)]
        xg_df.sort_index(inplace=True)

        X = xg_df.drop(columns=["company"])
        scaler = MinMaxScaler()
        X_scaled = scaler.fit_transform(X)
        pred_xgb = xgbr.predict(X_scaled)
        test = xg_df[["company", "close"]]
        test["prediction"] = pred_xgb
        test["gain_predicted"] = (test["prediction"] - test["close"]) / test["close"] * 100
        test.sort_values(by="gain_predicted", ascending=False, inplace=True)
        test.reset_index(inplace=True)
        test.drop(columns="date", inplace=True)
        test.set_index("company", inplace=True)

        return test

    

    xgbr = XGB_train_real()
    stocks = get_stocks()
    #stocks.to_csv("top_20.csv")
    return stocks



def plot_data(df, top):
    df["date"] = pd.to_datetime(df["date"])
    df.set_index("date", inplace=True)
    df.sort_index(inplace=True)
    df2 = df.loc["2020":]
    df3 = df2.drop(columns=["ma_60", "rsi", "macd", "volume", "dividends", "stock_splits", "marketcap", "gain_14", "loss_14", "rsi_14", "ema_12", "ema_26", "rsi_lag_5", "rsi_lag_10"])
    df3.reset_index(inplace=True)

    future = datetime.today().date() + timedelta(days=30)
    top["date"] = pd.to_datetime(future)

    top.drop(columns=["close", "gain_predicted"], inplace=True)
    top.rename(columns={"prediction": "close"}, inplace=True)

    merged = pd.concat([df3, top], ignore_index=True)
    merged.sort_values(by="date", inplace=True)

    return merged

In [12]:
tipps = run_xgb(ml1, ml1_training)
